In [31]:
import pandas as pd
import psycopg2 as pc2

In [32]:
data = pd.read_csv('https://www.dropbox.com/s/srpwa9lxhool57c/subset.csv?dl=1')
data = data.drop('Unnamed: 0', axis=1)
data.head()

/var/folders/p9/d79c5zx13b377v_v34ldc8zh0000gn/T/ipykernel_67223/2362573470.py:1: DtypeWarning: Columns (3,4,6,7,8,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('https://www.dropbox.com/s/srpwa9lxhool57c/subset.csv?dl=1')


,age_at_arrest,sex,race,encrypted_id,type_of_court,type_of_trial,type_of_counsel,final_pleading,year_of_sentencing,month_of_sentencing,elapsed_time_sentencing,sentence_min_years,sentence_max_years,sentence_type
0,21.0,NaN,NaN,PNGWXG318985,upper,NaN,NaN,nolo contendere,1990.0,8.0,8.0,NaN,NaN,NaN
1,21.0,NaN,NaN,KGQWIF309092,upper,NaN,NaN,nolo contendere,1990.0,10.0,11.0,NaN,NaN,NaN
2,22.0,NaN,NaN,NQWDUF319575,upper,NaN,NaN,nolo contendere,1990.0,7.0,7.0,NaN,NaN,NaN
3,21.0,NaN,NaN,BVRWXF318779,upper,NaN,NaN,nolo contendere,1990.0,4.0,100.0,NaN,NaN,NaN
4,30.0,NaN,NaN,DQXTRF409784,upper,NaN,NaN,nolo contendere,1990.0,6.0,285.0,0.0,0.0,one prison or jail sentence


In [42]:
params = dict(database='postgres',
              host='localhost',
              user='bloody_user',
              password='crime',
              port='5432')

connection = pc2.connect(**params)
cursor = connection.cursor()

In [43]:
# название колонок в датафрэйме
lol = ['encrypted_id', 'type_of_court', 'type_of_trial', 'type_of_counsel', 
       'final_pleading', 'sentence_type']
# название таблиц в базе
tab = ['offenders', 'court_type', 'trial_type', 'counselling_type', 
       'final_pleading', 'sentencing_type']
# название колонок для заполнения в базе
col = ['offenders_label', 'court_type_label', 'trial_type_label', 
       'counselling_type_label', 'final_pleading_label',
       'sentencing_type_label']
tol = list(zip(lol,tab,col))


In [44]:
def add_my_queue(to_where, to_col, queue):

    values_str = '(' + '), ('.join([f"'{v}'" for v in queue]) + ')' # добавить фильтр, который будет отсекать нан
    insert_query_DO = f"INSERT INTO {to_where} ({to_col}) VALUES {values_str}"
    cursor = connection.cursor()
    cursor.execute(insert_query_DO)
#
for t in tol:

    add_my_queue(t[1], t[2], data[f'{t[0]}'].dropna().unique())
    #
connection.commit()
connection.close()

In [39]:
data['type_of_court'].dropna().unique()

array(['upper', 'lower', 'other court'], dtype=object)